In [0]:
import pyspark.pandas as ps

In [0]:
path = 'dbfs:/FileStore/tables/dados_spotify_tratados/data.parquet'
df_data = ps.read_parquet(path)
df_data.info()

In [0]:
df_data = df_data.dropna()

In [0]:
df_data.info()

In [0]:
df_data['artists_song'] = df_data.artists + ' - ' + df_data.name

In [0]:
df_data.head(5)

In [0]:
df_data.info()

##Removendo colunas do tipo OBJECT

In [0]:
X = df_data.columns.to_list()
X.remove('artists')
X.remove('id')
X.remove('name')
X.remove('artists_song')
X.remove('release_date')
X

In [0]:
df_data = df_data.to_spark()

##Vetorização dos dados com VectorAssembler

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
dados_encoded_vector = VectorAssembler(inputCols=X, outputCol='features').transform(df_data)

In [0]:
dados_encoded_vector.select('features').show(truncate=False, n=5)

##Padronizando com StandardScaler

In [0]:
from pyspark.ml.feature import StandardScaler

In [0]:
scaler = StandardScaler(inputCol='features', outputCol='features_scaled')
model_scaler = scaler.fit(dados_encoded_vector)
dados_musicas_scaler = model_scaler.transform(dados_encoded_vector)

In [0]:
dados_musicas_scaler.select('features_scaled').show(truncate=False, n=5)

##Analisando quantidade de features existentes

In [0]:
k = len(X)
k

In [0]:
from pyspark.ml.feature import PCA

In [0]:
pca = PCA(k=k, inputCol='features_scaled', outputCol='pca_features') #criando o modelo
model_pca = pca.fit(dados_musicas_scaler) #Ajustando as tranformações
dados_musicas_pca = model_pca.transform(dados_musicas_scaler) #Transformando os dados

In [0]:
sum(model_pca.explainedVariance) * 100

In [0]:
lista_valores = [sum(model_pca.explainedVariance[0:i+1]) for i in range(k)]
lista_valores

##Criando PCA

In [0]:
import numpy as np

In [0]:
k = sum(np.array(lista_valores) <= 0.7) # definido o numero de componentes minimos para explicar 70% do nosso problema
k

In [0]:
#Criando o PCA com K = 6 
pca = PCA(k=k, inputCol='features_scaled', outputCol='pca_features') #criando o modelo
model_pca = pca.fit(dados_musicas_scaler) #Ajustando as tranformações
dados_musicas_pca_final = model_pca.transform(dados_musicas_scaler) #Transformando os dados

In [0]:
dados_musicas_pca_final.select('pca_features').show(truncate=False, n=5)

##Criando o Pipeline

In [0]:
from pyspark.ml import Pipeline

In [0]:
pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols=X, outputCol='features'),
                                StandardScaler(inputCol='features', outputCol='features_scaled'),
                                PCA(k=6, inputCol='features_scaled', outputCol='pca_features')])

In [0]:
model_pca_pipeline = pca_pipeline.fit(df_data)

In [0]:
projection = model_pca_pipeline.transform(df_data)

In [0]:
projection.select('pca_features').show(truncate=False, n=5)

##Utilizando o algoritmo K-means

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
SEED = 1224

In [0]:
kmeans = KMeans(k=50, featuresCol='pca_features', predictionCol='cluster_pca', seed=SEED)

In [0]:
modelo_kmeans = kmeans.fit(projection)

In [0]:
projetion_kmeans = modelo_kmeans.transform(projection)

In [0]:
projetion_kmeans.select(['pca_features', 'cluster_pca']).show()

##Analisando os Clusters

In [0]:
from pyspark.ml.functions import vector_to_array

In [0]:
projetion_kmeans = projetion_kmeans.withColumn('x', vector_to_array('pca_features')[0])\
                                   .withColumn('y', vector_to_array('pca_features')[1])

In [0]:
projetion_kmeans.select(['x', 'y', 'cluster_pca', 'artists_song']).show()

In [0]:
import plotly.express as px

In [0]:
fig = px.scatter(projetion_kmeans.toPandas(), x='x', y='y', color='cluster_pca', hover_data=['artists_song'])
fig.show()

##Usando o SilhouetteEvaluator no PySpark para avaliar a qualidade dos clusters criados pelo modelo K-means

In [0]:
import pyspark
print(pyspark.__version__)

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf, mean
from pyspark.sql.types import DoubleType
import numpy as np

# Dados de exemplo
data = spark.createDataFrame([
    (0, Vectors.dense([0.0, 0.0])),
    (1, Vectors.dense([1.0, 1.0])),
    (2, Vectors.dense([9.0, 8.0])),
    (3, Vectors.dense([8.0, 9.0]))
], ['id', 'features'])

# Treina o modelo
kmeans = KMeans(k=2, seed=1)
model = kmeans.fit(data)
transformed = model.transform(data)

# Extrai os centróides
centroids = model.clusterCenters()

# UDFs
def euclidean_distance(v1, v2):
    return float(np.linalg.norm(np.array(v1) - np.array(v2)))

@udf(DoubleType())
def distance_to_centroid(features, cluster_id):
    center = centroids[cluster_id]
    return euclidean_distance(features, center)

# Calcula a distância intra-cluster (a)
with_distance = transformed.withColumn("a", distance_to_centroid(col("features"), col("prediction")))

# Cross join para todas as combinações de pontos (menos eles mesmos)
cross = with_distance.alias("df1").join(with_distance.alias("df2"), col("df1.id") != col("df2.id"))

# Calcula distância entre todos os pontos
@udf(DoubleType())
def point_to_point_distance(p1, p2):
    return euclidean_distance(p1, p2)

cross = cross.withColumn("dist", point_to_point_distance(col("df1.features"), col("df2.features")))

# Calcula média da distância para pontos de outros clusters (b)
b_distances = cross.filter(col("df1.prediction") != col("df2.prediction")) \
    .groupBy("df1.id").agg(mean("dist").alias("b"))

# Junta a e b
silhouette_df = with_distance.join(b_distances, with_distance.id == b_distances["id"], "inner")

# Calcula silhouette score por ponto
@udf(DoubleType())
def silhouette(a, b):
    return (b - a) / max(a, b) if max(a, b) != 0 else 0.0

silhouette_df = silhouette_df.withColumn("silhouette", silhouette(col("a"), col("b")))

# Média geral
overall_score = silhouette_df.select(mean("silhouette")).first()[0]
print(f"Silhouette score: {overall_score:.4f}")


In [0]:
import matplotlib.pyplot as plt

from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf, rand, mean
from pyspark.sql.types import DoubleType
import numpy as np

# Função para calcular distância euclidiana
def euclidean_distance(v1, v2):
    return float(np.linalg.norm(np.array(v1) - np.array(v2)))

# Função que gera uma UDF com os centróides corretos para cada k
def make_distance_udf(centroids):
    @udf(DoubleType())
    def distance_to_centroid(features, cluster_id):
        if cluster_id < len(centroids):
            center = centroids[cluster_id]
            return euclidean_distance(features, center)
        else:
            return None
    return distance_to_centroid

# UDFs auxiliares
@udf(DoubleType())
def point_to_point_distance(p1, p2):
    return euclidean_distance(p1, p2)

@udf(DoubleType())
def silhouette(a, b):
    return (b - a) / max(a, b) if max(a, b) != 0 else 0.0

# Dados de exemplo com ID
data = spark.createDataFrame([
    (0, Vectors.dense([0.0, 0.0])),
    (1, Vectors.dense([1.0, 1.0])),
    (2, Vectors.dense([9.0, 8.0])),
    (3, Vectors.dense([8.0, 9.0]))
], ['id', 'features']).orderBy(rand())

# Avaliação do modelo K-means com diferentes k
silhouette_scores = []
k_values = range(2, 6)

for k in k_values:
    kmeans = KMeans(k=k, seed=1)
    model = kmeans.fit(data)
    transformed = model.transform(data)

    centroids = model.clusterCenters()
    distance_to_centroid = make_distance_udf(centroids)

    # Distância intra-cluster (a)
    with_a = transformed.withColumn("a", distance_to_centroid(col("features"), col("prediction")))

    # Distância inter-cluster (b)
    cross = with_a.alias("df1").join(with_a.alias("df2"), col("df1.id") != col("df2.id"))
    cross = cross.withColumn("dist", point_to_point_distance(col("df1.features"), col("df2.features")))
    b = cross.filter(col("df1.prediction") != col("df2.prediction")) \
             .groupBy("df1.id").agg(mean("dist").alias("b"))

    # Cálculo do silhouette por ponto
    silhouette_df = with_a.join(b, with_a.id == b["id"], "inner") \
                          .withColumn("silhouette", silhouette(col("a"), col("b")))

    # Score médio do cluster
    mean_score = silhouette_df.select(mean("silhouette")).first()[0]
    silhouette_scores.append(mean_score)

# Plot
plt.plot(k_values, silhouette_scores, '-o')
plt.xlabel('Número de Clusters')
plt.ylabel('Score de Silhueta')
plt.title('Silhouette Score vs. Número de Clusters')
plt.grid(True)
plt.show()


##Explorando modelo clusterizado

In [0]:
nome_musica = 'Taylor Swift - Blank Space'

In [0]:
cluster = projetion_kmeans.filter(projetion_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]
cluster

In [0]:
musicas_recomendadas = projetion_kmeans.filter(projetion_kmeans.cluster_pca == cluster)\
                                       .select('artists_song', 'id', 'pca_features')
musicas_recomendadas.show()

##Calculando distancia entre as musicas

In [0]:
componenetes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica)\
                                          .select('pca_features').collect()[0][0]
componenetes_musica                             

In [0]:
from scipy.spatial.distance import euclidean

In [0]:
def calcula_distance(value):
    return euclidean(componenetes_musica, value)

In [0]:
from pyspark.sql.types import FloatType
import pyspark.sql.functions as f

In [0]:
from scipy.spatial.distance import euclidean
from pyspark.sql.types import FloatType
import pyspark.sql.functions as f

In [0]:
def calcula_distance(value):
    return euclidean(componenetes_musica, value)

udf_calcula_distance = f.udf(calcula_distance, FloatType())

musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))

In [0]:
recomendadas = spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])

recomendadas.show()

##Criando uma unica função para o sistema de recomendação

In [0]:
def recomendador(nome_musica):
    cluster = projetion_kmeans.filter(projetion_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]
    musicas_recomendadas = projetion_kmeans.filter(projetion_kmeans.cluster_pca == cluster)\
                                       .select('artists_song', 'id', 'pca_features')
    componenetes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica)\
                                          .select('pca_features').collect()[0][0]

    def calcula_distance(value):
        return euclidean(componenetes_musica, value)

    udf_calcula_distance = f.udf(calcula_distance, FloatType())

    musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))

    recomendadas = spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])

    return recomendadas

In [0]:
df_recomedada = recomendador('Taylor Swift - Blank Space')
df_recomedada.show()

##Finalizando o sistema de recomendação

In [0]:
!pip install spotipy

In [0]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials

In [0]:
scope = "user-library-read playlist-modify-private"

OAuth = SpotifyOAuth(
        scope=scope,         
        redirect_uri='http://localhost:5000/callback',
        client_id = '203f92e3fae346d49ccef8b7f88ab809',
        client_secret = '2d3d7397b01944ec9792ca6eeb798d99')

In [0]:
client_credentials_manager = SpotifyClientCredentials(client_id = '203f92e3fae346d49ccef8b7f88ab809',
                                                      client_secret = '2d3d7397b01944ec9792ca6eeb798d99')

sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [0]:
id = projetion_kmeans.filter(projetion_kmeans.artists_song == nome_musica).select('id').collect()[0][0]
id

In [0]:
sp.track(id)

In [0]:
df_recomedada.select('id')

In [0]:
playlist_id = df_recomedada.select('id').collect()
playlist_id

In [0]:
playlist_track = []
for id in playlist_id:
    playlist_track.append(sp.track(id[0]))

In [0]:
def recomendador(nome_musica):
    cluster = projetion_kmeans.filter(projetion_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]
    musicas_recomendadas = projetion_kmeans.filter(projetion_kmeans.cluster_pca == cluster)\
                                       .select('artists_song', 'id', 'pca_features')
    componenetes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica)\
                                          .select('pca_features').collect()[0][0]

    def calcula_distance(value):
        return euclidean(componenetes_musica, value)

    udf_calcula_distance = f.udf(calcula_distance, FloatType())

    musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))

    recomendadas = spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])

    id = projetion_kmeans.filter(projetion_kmeans.artists_song == nome_musica).select('id').collect()[0][0]


    playlist_id = recomendadas.select('id').collect()

    playlist_track = []

    for id in playlist_id:
        playlist_track.append(sp.track(id[0]))

    return len(playlist_track)

In [0]:
recomendador('Taylor Swift - Blank Space')

In [0]:
!pip install scikit-image

In [0]:
import matplotlib.pyplot as plt
from skimage import io

nome_musica = 'Taylor Swift - Blank Space'

id = projetion_kmeans\
          .filter(projetion_kmeans.artists_song == nome_musica)\
          .select('id').collect()[0][0]

track = sp.track(id)

url = track["album"]["images"][1]["url"]
name = track["name"]

image = io.imread(url)
plt.imshow(image)
plt.xlabel(name, fontsize = 10)
plt.show()

In [0]:
import matplotlib.pyplot as plt
from skimage import io

def visualize_songs(name,url):

    plt.figure(figsize=(15,10))
    columns = 5
    for i, u in enumerate(url):
        ax = plt.subplot(len(url) // columns + 1, columns, i + 1)
        image = io.imread(u)
        plt.imshow(image)
        ax.get_yaxis().set_visible(False)
        plt.xticks(color = 'w', fontsize = 0.1)
        plt.yticks(color = 'w', fontsize = 0.1)
        plt.xlabel(name[i], fontsize = 10)
        plt.tight_layout(h_pad=0.7, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)
        plt.grid(visible=None)
    plt.show()

In [0]:
playlist_id = df_recomedada.select('id').collect()

name = []
url = []
for i in playlist_id:
    track = sp.track(i[0])
    url.append(track["album"]["images"][1]["url"])
    name.append(track["name"])

In [0]:
visualize_songs(name,url)

In [0]:
def recomendador(nome_musica):
  # Calcula musicas recomendadas
    cluster = projetion_kmeans.filter(projetion_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]
    musicas_recomendadas = projetion_kmeans.filter(projetion_kmeans.cluster_pca == cluster)\
                                       .select('artists_song', 'id', 'pca_features')
    componenetes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica)\
                                          .select('pca_features').collect()[0][0]

    def calcula_distance(value):
        return euclidean(componenetes_musica, value)

    udf_calcula_distance = f.udf(calcula_distance, FloatType())

    musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))

    recomendadas = spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])

  #Pegar informações da API

    playlist_id = recomendadas.select('id').collect()

    name = []
    url = []
    for i in playlist_id:
        track = sp.track(i[0])
        url.append(track["album"]["images"][1]["url"])
        name.append(track["name"])

  #Plotando capas 

    plt.figure(figsize=(15,10))
    columns = 5
    for i, u in enumerate(url):
        ax = plt.subplot(len(url) // columns + 1, columns, i + 1)
        image = io.imread(u)
        plt.imshow(image)
        ax.get_yaxis().set_visible(False)
        plt.xticks(color = 'w', fontsize = 0.1)
        plt.yticks(color = 'w', fontsize = 0.1)
        plt.xlabel(name[i], fontsize = 10)
        plt.tight_layout(h_pad=0.7, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)
        plt.grid(visible=None)
    plt.show()

In [0]:
recomendador('Taylor Swift - Blank Space')